<a href="https://colab.research.google.com/github/Porto-code/PhiFlow/blob/master/RPv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1
Start Integration with TensorForce Library

In [2]:
pip install --user tf-agents[reverb]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.4/624.4 kB 54.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 93.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 35.7 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.23.0-py3-none-any.whl size=697632 sha256=16ecaa0da34aece66f795ef907c3c05292fa7bdf1192690d79c42438661366c0

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
from gym import logger, spaces

from math import *
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import reverb

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts
from tf_agents.agents.reinforce import reinforce_agent
from tf_agents.drivers import py_driver
from tf_agents.networks import actor_distribution_network
from tf_agents.policies import py_tf_eager_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.specs import tensor_spec
from tf_agents.trajectories import trajectory
from tf_agents.utils import common
from tf_agents.policies import policy_saver

try:
  from google.colab import files
except ImportError:
  files = None


In [4]:
##Parameters

chord           = 2
density_air     = 1.225
Initial_Vel     = 50
mass            = 100
gravity         = 10
xcg             = chord*0.4
xc_4            = chord*0.25
deltaTime       = 0.1
TimeF           = 5


In [5]:
#functions
def lift(Velocity, alpha, u, NewTime):
    if NewTime == 0:
      L = 0.5 * density_air *chord* (Velocity**2) *2*pi * (alpha) + 0.5 * density_air * (Velocity)**2 *chord * 2*pi * u * 0
    else:
      tau = 2 * Velocity * NewTime / chord
      L = 0.5 * density_air *chord* (Velocity**2) *2*pi * (alpha) + 0.5 * density_air * (Velocity)**2 *chord * 2*pi * u * (tau +2)/(tau+4)
    return L

def ODE_airfoil(x,NewTime,u,Velocity,alpha):
    #x(0) z
    #x(1) alpha
    #xpp=[0,0]
    #xpp[0] = lift(Velocity,x[1],u,NewTime) - mass * gravity
    #xpp[1] = 2*pi*(x[1]+u)*(xcg-xc_4)

    xpp    = np.array([0,0],dtype=np.float32)
    xpp[0] = x[1]
    xpp[1] = (lift(Velocity,alpha,u,NewTime) - mass * gravity)/mass
    return xpp

In [6]:
#Definitions
interaction    = 0
N_interactions = TimeF/deltaTime
deltaTime_vec  = np.linspace(0, deltaTime, 5)
alpha0         = mass*gravity/(0.5 * density_air *chord* (Initial_Vel**2) *2*pi )



In [30]:
class TheodorsenModelRL(py_environment.PyEnvironment):
#py_environment.PyEnvironment
    def __init__(self):
        self.gravity         = 10
        self.chord           = 2
        self.mass            = 100
        self.density_air     = 1.225
        self.initial_Vel     = 50
        self.alpha           = self.mass*self.gravity/(0.5 * self.density_air *self.chord* (self.initial_Vel**2) *2*pi )
        self.tau = 0.1  # seconds between state updates
        self.z_threshold = 2
#        self.action_space = spaces.Discrete(5)
#        self.interaction = 0
        self.TimeF      = 3
        self.itermax      = self.TimeF/self.tau
        #_state = [z,zd,InternalTime, Vel,aux, interaction]
        self._state = [0, 0, 0 , self.initial_Vel,0]
        high = np.array(
            [
                np.finfo(np.float32).max,
                np.finfo(np.float32).max,
                self.TimeF,
                self.initial_Vel * 1.5,
                pi/20,
                self.itermax + 1
            ],
            dtype=np.float32,
        )
        low = np.array(
            [
                -np.finfo(np.float32).max,
                -np.finfo(np.float32).max,
                0,
                self.initial_Vel * 0.5,
                -pi/20,
                0
            ],
            dtype=np.float32,
        )
        self._action_spec      = array_spec.BoundedArraySpec(shape=(), dtype=np.float32, minimum=-pi/20, maximum=pi/20, name='actions')
        self._observation_spec = array_spec.BoundedArraySpec(shape=(1,6), dtype=np.float32,minimum=low, maximum=high, name='observation')
        self._episode_ended = False


    def action_spec(self):
      return self._action_spec
    def observation_spec(self):
      return self._observation_spec

    def _reset(self):
        self._state = [0, 0, 0 , self.initial_Vel, 0 , 0]
        self._episode_ended = False
        return ts.restart(np.array([self._state], dtype=np.float32))

    def _step(self, actions):
      if self._episode_ended:
      # The last action ended the episode. Ignore the current action and start
      # a new episode.
        return self.reset()
      z,zdot, NewTime, Velocity,aux, interaction = self._state
      y0  =[z,zdot]
      if aux == actions:
            NewTime = self.tau*interaction
      else:
            NewTime = 0
            self.alpha = self.alpha+actions

      NewTime_vec            = np.linspace(NewTime, NewTime + self.tau, 5)
      ODEresult              = odeint(ODE_airfoil, y0, NewTime_vec, args=(actions, Velocity,self.alpha))
      z_f                    = ODEresult[-1, 0]
      zd_f                   = ODEresult[-1, 1]
      aux                    = actions
      interaction            = interaction + 1
      self._episode_ended    = bool(z_f < -self.z_threshold or z_f > self.z_threshold or interaction>self.itermax)
      self._state = [z_f,zd_f,NewTime+self.tau, self.initial_Vel,aux, interaction]

      if self._episode_ended:
        reward = - abs(zd_f)/self.initial_Vel - abs(z_f)/self.chord
        return ts.termination(np.array([self._state], dtype=np.float32), reward)
      else:
        reward                 = - abs(zd_f)/self.initial_Vel
        return ts.transition(np.array([self._state], dtype=np.float32), reward, discount=1.0)


In [35]:

env = TheodorsenModelRL()
tf_env = tf_py_environment.TFPyEnvironment(env)




In [31]:
get_new_action = np.array([0], dtype=np.float32)

environment = TheodorsenModelRL()
time_step = environment.reset()
cumulative_reward = time_step.reward

for i in range(3):
  time_step = environment.step(get_new_action)
  print(time_step)
  cumulative_reward += time_step.reward

print('Final Reward = ', cumulative_reward)

TimeStep(
{'discount': array(1., dtype=float32),
 'observation': array([[ 0. ,  0. ,  0.1, 50. ,  0. ,  1. ]], dtype=float32),
 'reward': array(-0., dtype=float32),
 'step_type': array(1, dtype=int32)})
TimeStep(
{'discount': array(1., dtype=float32),
 'observation': array([[ 0. ,  0. ,  0.2, 50. ,  0. ,  2. ]], dtype=float32),
 'reward': array(-0., dtype=float32),
 'step_type': array(1, dtype=int32)})
TimeStep(
{'discount': array(1., dtype=float32),
 'observation': array([[ 0. ,  0. ,  0.3, 50. ,  0. ,  3. ]], dtype=float32),
 'reward': array(-0., dtype=float32),
 'step_type': array(1, dtype=int32)})
Final Reward =  0.0


<ipython-input-30-363dd2827247>:82: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  return ts.transition(np.array([self._state], dtype=np.float32), reward, discount=1.0)


In [34]:
num_iterations = 250 # @param {type:"integer"}
collect_episodes_per_iteration = 2 # @param {type:"integer"}
replay_buffer_capacity = 2000 # @param {type:"integer"}

fc_layer_params = (100,)

learning_rate = 1e-3 # @param {type:"number"}
log_interval = 25 # @param {type:"integer"}
num_eval_episodes = 10 # @param {type:"integer"}
eval_interval = 50 # @param {type:"integer"}

actor_net = actor_distribution_network.ActorDistributionNetwork(
    tf_env.observation_spec(),
    tf_env.action_spec(),
    fc_layer_params=fc_layer_params)

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

tf_agent = reinforce_agent.ReinforceAgent(
    tf_env.time_step_spec(),
    tf_env.action_spec(),
    actor_network=actor_net,
    optimizer=optimizer,
    normalize_returns=True,
    train_step_counter=train_step_counter)
tf_agent.initialize()

eval_policy = tf_agent.policy
collect_policy = tf_agent.collect_policy

In [33]:
def compute_avg_return(environment, policy, num_episodes):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return

def collect_episode(environment, policy, num_episodes):

  driver = py_driver.PyDriver(
    environment,
    py_tf_eager_policy.PyTFEagerPolicy(
      policy, use_tf_function=True),
    [rb_observer],
    max_episodes=num_episodes)
  initial_time_step = environment.reset()
  driver.run(initial_time_step)

In [36]:
table_name = 'uniform_table'
replay_buffer_signature = tensor_spec.from_spec(
      tf_agent.collect_data_spec)
replay_buffer_signature = tensor_spec.add_outer_dim(
      replay_buffer_signature)
table = reverb.Table(
    table_name,
    max_size=replay_buffer_capacity,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1),
    signature=replay_buffer_signature)

reverb_server = reverb.Server([table])

replay_buffer = reverb_replay_buffer.ReverbReplayBuffer(
    tf_agent.collect_data_spec,
    table_name=table_name,
    sequence_length=None,
    local_server=reverb_server)

rb_observer = reverb_utils.ReverbAddEpisodeObserver(
    replay_buffer.py_client,
    table_name,
    replay_buffer_capacity
)

In [37]:
try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
tf_agent.train = common.function(tf_agent.train)

# Reset the train step
tf_agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(tf_env, tf_agent.policy, num_eval_episodes)
returns = [avg_return]

for _ in range(num_iterations):

  # Collect a few episodes using collect_policy and save to the replay buffer.
  collect_episode(
      TheodorsenModelRL(), tf_agent.collect_policy, collect_episodes_per_iteration)

  # Use data from the buffer and update the agent's network.
  iterator = iter(replay_buffer.as_dataset(sample_batch_size=1))
  trajectories, _ = next(iterator)
  train_loss = tf_agent.train(experience=trajectories)

  replay_buffer.clear()

  step = tf_agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss.loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(TheodorsenModelRL(), tf_agent.policy, num_eval_episodes)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)

step = 25: loss = -0.2667805254459381
step = 50: loss = 0.1544724851846695
step = 50: Average Return = -3.2769603431224823
step = 75: loss = 0.0467832125723362
step = 100: loss = -0.14544859528541565
step = 100: Average Return = -3.2769683256745337
step = 125: loss = -0.16253730654716492
step = 150: loss = -0.015782838687300682
step = 150: Average Return = -3.276972709596157
step = 175: loss = -0.01562216691672802
step = 200: loss = -0.054035384207963943
step = 200: Average Return = -3.2769765190780165
step = 225: loss = -0.17646464705467224
step = 250: loss = -0.5310866832733154
step = 250: Average Return = -3.276980097591877
